In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
df = pd.read_csv('Pregnancy Risk.csv')
df.shape

(1229, 16)

In [2]:
df.sample(5)

,Unnamed: 0,Age,Height,Weight,BMI,Blood Group ( is Negative?),Delivery Type,Systolic Blood Pressure,Diastolic Blood Pressure,Edema,Anemia,Jaundice,Diabetes,Risk,Para,Gravida
336,336,25,1.422,53.0,26.20,No,No_delivary,120,80,Normal,Normal,No,No,Yes,0,1
961,961,18,1.524,57.6,24.80,No,No_delivary,90,60,Normal,Modarate,No,No,Yes,0,1
1017,1017,18,1.549,44.0,18.33,No,No_delivary,130,90,Normal,Modarate,No,No,No,0,1
549,549,35,1.473,60.0,27.65,No,Normal,100,60,Normal,Modarate,No,No,Yes,1,2
108,108,20,1.524,48.7,20.97,No,No_delivary,100,80,Normal,Normal,No,No,No,0,1


In [3]:
df1 = df.drop(['Unnamed: 0'],axis=1)
df1.head(5)

,Age,Height,Weight,BMI,Blood Group ( is Negative?),Delivery Type,Systolic Blood Pressure,Diastolic Blood Pressure,Edema,Anemia,Jaundice,Diabetes,Risk,Para,Gravida
0,25,1.600,50.7,19.80,No,No_delivary,110,90,Normal,Normal,No,No,No,0,1
1,30,1.524,86.3,37.16,No,Normal,110,80,Normal,Normal,No,No,No,1,2
2,26,1.549,58.8,24.49,No,Normal,120,80,Normal,Normal,No,No,No,0,1
3,26,1.549,86.8,36.16,No,Normal,120,80,Normal,Normal,No,Yes,Yes,1,2
4,36,1.422,85.8,42.90,No,Normal,110,80,Normal,Normal,No,No,Yes,0,1


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
col = ['Blood Group ( is Negative?)','Jaundice','Diabetes','Risk']
for x in col:
    df1[x] = encoder.fit_transform(df[x])
df1.head(5)

,Age,Height,Weight,BMI,Blood Group ( is Negative?),Delivery Type,Systolic Blood Pressure,Diastolic Blood Pressure,Edema,Anemia,Jaundice,Diabetes,Risk,Para,Gravida
0,25,1.600,50.7,19.80,0,No_delivary,110,90,Normal,Normal,0,0,0,0,1
1,30,1.524,86.3,37.16,0,Normal,110,80,Normal,Normal,0,0,0,1,2
2,26,1.549,58.8,24.49,0,Normal,120,80,Normal,Normal,0,0,0,0,1
3,26,1.549,86.8,36.16,0,Normal,120,80,Normal,Normal,0,1,1,1,2
4,36,1.422,85.8,42.90,0,Normal,110,80,Normal,Normal,0,0,1,0,1


In [5]:
df2 = pd.get_dummies(df1,drop_first=True)
df2.sample(2)

,Age,Height,Weight,BMI,Blood Group ( is Negative?),Systolic Blood Pressure,Diastolic Blood Pressure,Jaundice,Diabetes,Risk,Para,Gravida,Delivery Type_No_delivary,Delivery Type_Normal,Edema_Modarate,Edema_Normal,Anemia_Modarate,Anemia_Normal
626,23,1.549,42.0,17.50,0,90,60,0,0,1,1,2,0,1,0,1,0,1
977,18,1.524,47.5,20.45,0,90,70,0,0,0,0,1,1,0,0,1,0,1


In [6]:
X = df2.drop(['Risk','Height','Weight'],axis=1)
y = df2.Risk

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
len(X_train)

860

In [9]:
len(X_test)

369

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [27]:
model_param = {
    
    'LogisticRegression' : {
        'model' : LogisticRegression(solver='liblinear',multi_class='auto'),
        'params' : {
            'C': np.arange(1,20),
            'penalty':['l1', 'l2', 'elasticnet',None],
            'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
        }
    },
    
    'Decision_Tree' : {
        'model' : DecisionTreeClassifier(),
        'params' : {
            'criterion':['gini','entropy','log_loss'],
            'splitter': ['best','random'],
            'max_depth': np.arange(1,20)
        }
    },
    
    'SVM' : {
        'model' : SVC(),
        'params' : {
            'kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
            'gamma': ['scale', 'auto']
        }
    },
    
    'Random_Forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators': np.arange(5,30,5),
            'criterion':['gini','entropy','log_loss'],
            'max_depth': np.arange(1,20)
        }
    },
    
    'KNN' : {
        'model' : KNeighborsClassifier(),
        'params' : {
            'n_neighbors': np.arange(5,10),
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
        }
    },
    
    'GaussianNB':{
        'model' : GaussianNB(),
        'params' : { }
    },
    
    'MultinomialNB':{
        'model' : MultinomialNB(),
        'params' : {
            'alpha': np.arange(1,10)
        }
    }
    
}

In [12]:
model_params = {           
    'Random_Forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators': np.arange(5,30,5),
            'criterion':['gini','entropy','log_loss'],
            'max_depth': np.arange(1,20)
        }
    }
}

In [13]:
model_params.items()

dict_items([('Random_Forest', {'model': RandomForestClassifier(), 'params': {'n_estimators': array([ 5, 10, 15, 20, 25]), 'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])}})])

In [14]:
score=[]
from sklearn.metrics import recall_score,precision_score,f1_score,auc,confusion_matrix
for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=10,return_train_score=False)
    clf.fit(X,y)
    score.append({
        'model':model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_,
    })
pd.DataFrame(score)

,model,best_score,best_params
0,Random_Forest,0.895009,"{'criterion': 'gini', 'max_depth': 16, 'n_esti..."


In [15]:
score

[{'model': 'Random_Forest',
  'best_score': 0.8950086632013861,
  'best_params': {'criterion': 'gini', 'max_depth': 16, 'n_estimators': 15}}]

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

recall = cross_val_score(RandomForestClassifier(), X, y, cv=15, scoring='recall')
print('Recall: ', np.average(recall))
precision = cross_val_score(RandomForestClassifier(), X, y, cv=15, scoring='precision')
print('Precision: ',np.average(precision))
f1 = cross_val_score(RandomForestClassifier(), X, y, cv=15, scoring='f1')
print('F1: ',round((np.average(f1)*100)),'%')

Recall:  0.7528888888888889
Precision:  0.8763961448481571
F1:  80 %


In [29]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

grid_search_rfc = GridSearchCV(RandomForestClassifier(), param_grid = {},cv=15, scoring = scoring, refit='f1')
grid_search_rfc.fit(X, y)

GridSearchCV(cv=15, estimator=RandomForestClassifier(), param_grid={},
             refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

In [30]:
pd.DataFrame(grid_search_rfc.cv_results_)[['params','mean_test_precision','mean_test_recall','mean_test_f1']]

,params,mean_test_precision,mean_test_recall,mean_test_f1
0,{},0.884171,0.758667,0.81382


In [19]:
grid_search_rfc.cv_results_['mean_test_precision'].mean()

0.893001318078694

In [20]:
for model_name,mp in model_params.items():
    clf = grid_search = GridSearchCV(
    estimator=mp['model'],
    param_grid=mp['params'],
    scoring=['accuracy', 'f1', 'precision', 'recall'],
    refit="accuracy",  # Or any other value from `scoring` list
)
    clf.fit(X,y)
    print(clf.best_score_)
    print(clf.best_params_)
    print(clf.cv_results_['mean_test_precision'].mean())
    print(clf.cv_results_['mean_test_recall'].mean())
    print(clf.cv_results_['mean_test_f1'].mean())
#     print(pd.DataFrame(clf.cv_results_)[['params','mean_test_precision','mean_test_recall','mean_test_f1']])

0.8990774846523977
{'criterion': 'gini', 'max_depth': 18, 'n_estimators': 25}
nan
nan
nan
